In [13]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import os

addr = pd.concat([
    pd.read_csv(f'addr/{f}').rename(columns={
        'replace_name':'name'}) for f in [
            'non_exchange_address.csv', 
            'exchange_address2.csv',
            'address_kor_exc.csv',
        ]
])
addr.name = addr.name.str.lower()
addr.name = addr.name.str.replace('_deposit', '')
addr.name = addr.name.str.replace('defi-', '')
addr.exchange_name = addr.exchange_name.str.lower()


for v in ['isContract', 'isWhale']:
    addr[v] = False
for name, elements in {
   'network': ['eth', 'trx']
}.items():
    addr[name] = None
    addr[name] = addr[name].astype(
        CategoricalDtype(categories=elements, ordered=False)
    )

for name, elements in {
    "serviceType": ['CeFi', 'DeFi', 'Gov', 'Other'],
    'sub1Service': [
        'exc', 'lend', 'custody', 'fund', 
        'bridge', 'agg', 'yield', 'oracle', 'LP',
        'dao', 
        'susp', 'MEV', 'treasury', 'chain',  'etc',
    ],
    'walletPurpose': ['Staking', 'Infra', 'Vault', 'Gov', 'Other'],
    'sub1Wallet':[
        'soloStaker', 'stakingPool', 
        'multisig', 'mint', 'validator', 
        'hot', 'cold', 'liqPool', 'deposit',
        'voter', 'delegate', 'delegator',
        'proxy', 'router', 'resolver', 'marketMaker',
    ],
}.items():
    addr[name] = None
    addr[name] = addr[name].astype(
        CategoricalDtype(categories=elements, ordered=False)
    )

is_contain_in_name = lambda x: addr.name.str.contains(x, regex=True, na=False)
is_end = lambda x: addr.name.str.endswith(x, na=False)
    
    
addr.loc[addr.name.isna(), 'serviceType'] = "CeFi"
addr.loc[addr.name.isna(), 'sub1Service'] = "exc"
addr.loc[~addr.name.isna(), 'serviceType'] = "DeFi"
addr.loc[addr.name.isna(), 'name'] = addr.loc[addr.name.isna(), 'exchange_name']

addr.loc[addr.address.str.contains(
    r'(null|0000000000000000000000000000)', regex=True, na=False), 'name'] = 'trash'

for k, v in {
    'treasury': [r'(circle|tether)'],
    'etc': [r'(opensea|metamask)', r'(null|0000000000000000000000000000)']
}.items():
    for vv in v:        
        addr.loc[is_contain_in_name(vv), 'serviceType'] = 'Other'
        addr.loc[is_contain_in_name(vv), 'sub1Service'] = k

addr.loc[is_end('_network'), 'serviceType'] = 'Other'
addr.loc[is_end('_network'), 'sub1Service'] = 'chain'
addr.loc[addr.name.isin(['base', 'optimism', 'polygon']), 'serviceType'] = 'Other'
addr.loc[addr.name.isin(['base', 'optimism', 'polygon']), 'sub1Service'] = 'chain'


addr.loc[is_end('custody'), 'serviceType'] = 'DeFi'
addr.loc[is_end('custody'), 'sub1Service'] = 'custody'
addr.loc[addr.name=='bridgers', 'serviceType'] = 'DeFi'
addr.loc[addr.name=='bridgers', 'sub1Service'] = 'bridge'



addr = addr.drop(columns=['exchange_name'])

addr.loc[addr.name=="bistamp", 'name'] = 'bitstamp'
addr.loc[addr.name=="smartexchangerouter", 'name'] = 'suniosmartexchangerouter'
addr.loc[addr.name=="sky(makerdao)", 'name'] = 'makerdao'
addr.loc[addr.sub1Service=='chain', 'name'] = addr.loc[addr.sub1Service=='chain', 'name'].str.replace('_network', '')
addr.loc[addr.sub1Service=='custody', 'name'] = addr.loc[addr.sub1Service=='custody', 'name'].str.replace('custody', '')


addr.loc[addr.address.str[:2]=='0x', 'network'] = 'eth'
addr.loc[addr.address.str[0]=='T', 'network'] = 'trx'
addr.address = addr.address.str.lower()
addr = addr.drop_duplicates()

addr

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14776\114014867.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  addr.loc[addr.address.str.contains(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14776\114014867.py:50: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_contain_in_name = lambda x: addr.name.str.contains(x, regex=True, na=False)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14776\114014867.py:50: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_contain_in_name = lambda x: addr.name.str.contains(x, regex=True, na=False)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14776\114014867.py:50: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually g

,address,name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet
0,0x55fe002aeff02f77364de339a1292923a15844b8,circle,False,False,eth,Other,treasury,NaN,NaN
1,0xc4922d64a24675e16e1586e3e3aa56c06fabe907,circletokenminter,False,False,eth,Other,treasury,NaN,NaN
2,0x111111125421ca6dc452d289314280a0f8842a65,1inch,False,False,eth,DeFi,NaN,NaN,NaN
3,0x1111111254eeb25477b68fb85ed929f73a960582,1inch,False,False,eth,DeFi,NaN,NaN,NaN
4,0x3451b6b219478037a1ac572706627fc2bda1e812,1inch,False,False,eth,DeFi,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14965,0xd388889dea826e64757abfeeda952ca2a6c6e6c2,bithumb,False,False,eth,DeFi,NaN,NaN,NaN
14966,0x6720e2e748baec9e1f2efef4745fd86ec2d101b3,bithumb,False,False,eth,DeFi,NaN,NaN,NaN
14967,0xb67750b1745349fbd6dd6477fd5a05070d9e3e0c,bithumb,False,False,eth,DeFi,NaN,NaN,NaN
14968,0xb46a19d227bdb8a48fae93e1e9abd897f7c0fee1,bithumb,False,False,eth,DeFi,NaN,NaN,NaN


In [14]:
addr.name.unique()

array(['circle', 'circletokenminter', '1inch', 'aave', 'arbitrum',
       'bridgers', 'cowprotocol', 'curve', 'dodo', 'justlend', 'makerdao',
       'suniosmartexchangerouter', 'sunio', 'uniswap', 'justinsun',
       'trash', 'stusdt', 'tethermultisig', 'tethertresury',
       'swftblockchain', 'liquity', 'lido', 'nexo', 'looksrare', 'blur',
       'opensea', 'synapse_protocol', 'stargate', 'connext', 'wormhole',
       'hop_protocol', 'orbiter_finance', 'compound', 'paraswap',
       'metamask', 'lifi', 'sushiswap', 'contango', 'sudoswap', 'kyber',
       'balancer', '0x', 'pancakeswap', 'dln_trade', 'xy_finance',
       'shibaswap', 'solidly', 'squid', 'quickswap', 'rabbitx', 'socket',
       'axelar', 'layerzero', 'celer', 'pendle_finance', 'aura_finance',
       'yearnfinance', 'beefy_finance', 'rocket_pool', 'wintermute',
       'grayscale', 'base', 'optimism', 'polygon', '96ach', 'alphapo',
       'ascendex', 'binance', 'bingx', 'bitstamp', 'bitfinex', 'bitflyer',
       'bitget'

In [15]:
ad = pd.read_csv('addr/addresses_data_all.csv', usecols=[
    'address', 'chain', 'contract', 'isUserAddress', 'arkhamEntity_type', 'arkhamEntity_name', 'arkhamLabel_name', 'arkhamEntity_website'
]).drop_duplicates()
ad.address = ad.address.str.lower()

ad.arkhamEntity_type.unique()

array([nan, 'nft-marketplace', 'dex', 'misc', 'dex-aggregator', 'fund',
       'individual', 'cex', 'lending-centralized', 'fund-decentralized',
       'derivatives', 'custodian', 'gaming', 'lending-decentralized',
       'yield', 'government', 'real-world-assets', 'cdp',
       'blockchain-infra', 'dao', 'smart-contract-platform', 'gambling',
       'bridge', 'miner-validator', 'insurance',
       'crosschain-interoperability', 'charity', 'stablecoin', 'nft',
       'mev', 'staking', 'hacker', 'prediction-market', 'liquid-staking',
       'meme', 'ai', 'otc', 'blockchain-scaling', 'oracle', 'social'],
      dtype=object)

In [16]:


result = addr.merge(
    ad, on='address', how='outer',indicator=True
)
is_contain_in_type = lambda x: result.arkhamEntity_type.str.contains(x, na=False)
is_contain_in_name = lambda x: result.arkhamLabel_name.str.contains(x, na=False)



result.loc[result.address.str[:2]=='0x', 'network'] = 'eth'
result.loc[result.address.str[0]=='T', 'network'] = 'trx'


result.loc[result.contract==True, 'isContract'] = True # Token?
result.loc[result.arkhamLabel_name.str.contains(r'\([A-Z]+\)', regex=True, na=False), 'isContract'] = True
### Service Type
result.loc[is_contain_in_name('MEV'), 'name'] = 'MEV'
result.loc[is_contain_in_name('MEV'), 'serviceType'] = 'Other'
result.loc[is_contain_in_name('MEV'), 'sub1Service'] = 'MEV'
result.loc[is_contain_in_name('_User'), 'name'] = 'User'
result.loc[is_contain_in_name('_User'), 'serviceType'] = 'Other'
result.loc[is_contain_in_name('_User'), 'sub1Service'] = 'etc'
result.loc[result.isUserAddress==True, 'name'] = 'User'
result.loc[result.isUserAddress==True, 'serviceType'] = 'Other'
result.loc[result.isUserAddress==True, 'sub1Service'] = 'etc'
result.loc[is_contain_in_type('individual'), 'name'] = 'User'
result.loc[is_contain_in_type('individual'), 'serviceType'] = 'Other'
result.loc[is_contain_in_type('individual'), 'sub1Service'] = 'etc'



result.loc[is_contain_in_type('cex'), 'sub1Service'] = 'exc'
result.loc[is_contain_in_type('cex'), 'serviceType'] = 'CeFi'

result.loc[is_contain_in_type('yield'), 'sub1Service'] = 'yield'
result.loc[is_contain_in_type('yield'), 'serviceType'] = 'DeFi'

result.loc[is_contain_in_type('fund'), 'sub1Service'] = 'fund'
result.loc[is_contain_in_type('fund'), 'serviceType'] = 'CeFi'
result.loc[is_contain_in_type('fund-decentralized'), 'serviceType'] = 'DeFi'

result.loc[is_contain_in_type('lending'), 'sub1Service'] = 'lend'
result.loc[is_contain_in_type('lending'), 'serviceType'] = 'CeFi'
result.loc[is_contain_in_type('lending-decentralized'), 'serviceType'] = 'DeFi'


result.loc[is_contain_in_type('custodian'), 'sub1Service'] = 'custody'

result.loc[is_contain_in_type('dex'), 'sub1Service'] = 'exc'
result.loc[is_contain_in_type('dex'), 'serviceType'] = 'DeFi'
result.loc[is_contain_in_type('dex-aggregator'), 'sub1Service'] = 'agg'


result.loc[result.name.isna(), 'name'] = result.loc[result.name.isna(), 'arkhamEntity_name']




import numpy as np


purpose_choices = ['Staking', 'Vault', 'Infra', 'Gov', 'Other']
purpose_conditions = [
    result.arkhamLabel_name.str.contains(v, na=False, regex=True)
    for v in [
        r'(?:Staking)', r'(?:Wallet|Deposit|Pool|USD)',
        r'(?:Mint|Valid|Multisig|Miner)',  r'(?:Voter|Delegat)',
        r'(?:Proxy|Router|Resolver|Market Maker)'
    ]
]

sub_choices = ['cold', 'hot',  'liqPool', 'deposit', 
               'mint', 'multisig', 'validator',
               'proxy', 'router', 'resolver', 'marketMaker']
sub_conditions = [
    result.arkhamLabel_name.str.contains(pattern, na=False, regex=True)
    for pattern in [
        r'(?:Cold Wallet)', r'(?:Hot Wallet)', r'(?:Pool|USD)', r'(?:Deposit)',
        r'(?:Mint)', r'(?:Multisig)', r'(?:Validat|Miner)',
        r'(?:Proxy)', r'(?:Router)', r'(?:Resolver)', r'(?:Market Maker)',
    ]
]

result['walletPurpose'] = np.select(purpose_conditions, purpose_choices, default='')
result['sub1Wallet'] = np.select(sub_conditions, sub_choices, default='')
print(result._merge.value_counts())

_merge
right_only    209229
left_only      17975
both             269
Name: count, dtype: int64


In [17]:
result

,address,name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet,chain,isUserAddress,contract,arkhamEntity_name,arkhamEntity_type,arkhamEntity_website,arkhamLabel_name,_merge
0,0x0000000000000000000000000000000000000000,trash,False,False,eth,DeFi,NaN,,,ethereum,False,False,NaN,NaN,NaN,Null Address,both
1,0x0000000000000000000000000000000000000003,trash,False,False,eth,DeFi,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,0x0000000000000000000000000000000000000005,trash,False,False,eth,DeFi,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,0x0000000000000000000000000000000000000006,trash,False,False,eth,DeFi,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0x0000000000000000000000000000000000000008,trash,False,False,eth,DeFi,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227468,tz1ssapyhknwavlca6p2qgvzkhtdk6nkxa,htx,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227469,tzbaacgmaa84rpwa1aswl6emc7b9gmbdrk,bybit,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227470,tzbrpsrqsqyxksiupuz7r4ldbokndqar5y,unknownexchange5,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227471,tzhw3pje6voyv8nwaaeukqwnre3bvu7u8n,bitget,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [18]:
result.loc[result.name.str.contains('unknown', na=False)]

,address,name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet,chain,isUserAddress,contract,arkhamEntity_name,arkhamEntity_type,arkhamEntity_website,arkhamLabel_name,_merge
227295,tes1pgfote8fmrbweyjgxqmrb2uss74nvm,unknownexchange3,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227297,tf6814bxaomnu6s7xzztqb7jbqxkkwovlz,unknownexchange1,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227332,tjwt1j97d2czjqyhvuqrq4ytmymfvmypxb,unknownexchange2,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227337,tkfz7pu3jtxwrfhzveozqjxsb3rfuuhge2,unknownexchange8,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227355,tmtjlkzfwcqckkttou5fexz3eygomwaz55,unknownexchange7,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227371,tps3x9wzmu63ommngrsuxqn9e2af4qtrch,unknownexchange4,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227390,trytwabkiapvme8u5hhkp2ffdmaydlsbuy,unknownexchange6,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227429,tvkhba3eh4jf5kq7uykfyz7asfbkgtjmlm,unknownexchange10,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227437,twd1bpfysqrwctddshor31vqyxqd286mng,unknownexchange9,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
227470,tzbrpsrqsqyxksiupuz7r4ldbokndqar5y,unknownexchange5,False,False,trx,CeFi,exc,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [19]:
result[result.isContract==True][['name', 'isContract', 'isWhale', 'network', 'serviceType',
       'sub1Service', 'walletPurpose', 'sub1Wallet']].value_counts(dropna=False).to_frame()

count
name                   isContract isWhale network serviceType sub1Service walletPurpose sub1Wallet       
NaN                    True       NaN     eth     NaN         NaN         Vault         deposit      7473
                                                                          Other         proxy        1742
                                                                                                     1627
Uniswap                True       NaN     eth     DeFi        exc         Vault         liqPool       610
NaN                    True       NaN     eth     NaN         NaN         Vault                       167
...                                                                                                   ...
Metalabel              True       NaN     eth     NaN         NaN                                       1
MetaMask               True       NaN     eth     DeFi        agg         Other         proxy           1
                                                                                                        1
Meson Finance          True       NaN     eth     NaN         NaN         Other         proxy           1
Alpha (1inch Resolver) True       NaN     eth     DeFi        exc         Other         resolver        1

[580 rows x 1 columns]

In [20]:
result = result[[
    'address', 'name', 'isContract', 'isWhale', 'network', 'serviceType',
       'sub1Service', 'walletPurpose', 'sub1Wallet',
]].reset_index(drop=True)
print(result.shape)
result.to_csv('addr_info.csv')
result[[
    'serviceType', 'sub1Service', 'walletPurpose', 'sub1Wallet'
]].value_counts(dropna=False).to_frame().sort_index()

(227473, 9)


count
serviceType sub1Service walletPurpose sub1Wallet        
CeFi        exc                                     1253
                        Other         proxy            3
                        Vault                          4
                                      cold             6
                                      deposit          7
...                                                  ...
NaN         NaN         Vault                        168
                                      deposit     106421
                                      hot              9
                                      liqPool         75
                                      proxy            3

[86 rows x 1 columns]

In [21]:
result[(result.isContract==True) & (~result.serviceType.isna())]

,address,name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet
7,0x000000000000006f6502b7f2bbac8c30a3f67e9a,MEV,True,NaN,eth,Other,MEV,,
10,0x0000000000000eb4ec62758aae93400b3e5f7f18,MEV,True,NaN,eth,Other,MEV,,
12,0x0000000000005117dd3a72e64a705198753fdd54,MEV,True,NaN,eth,Other,MEV,,
13,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,MEV,True,NaN,eth,Other,MEV,,
15,0x00000000000123685885532dcb685c442dc83126,MEV,True,NaN,eth,Other,MEV,,
...,...,...,...,...,...,...,...,...,...
211523,0xfefb05eb5abc0c378120b93ff37cec8af4872a18,DefinitiveFi,True,NaN,eth,DeFi,agg,Vault,liqPool
211959,0xff9127f3d47eadc134a147a9f3ae996444895348,Enzyme Finance,True,NaN,eth,DeFi,fund,Other,proxy
211990,0xff9b21c3bfa4bce9b20b55fed56d102ced48b0f6,Symbiosis Finance,True,NaN,eth,DeFi,exc,,
212119,0xffc9efd66926e3023a4cbd20145ca48f52dff2d9,Uniswap,True,NaN,eth,DeFi,exc,Vault,liqPool


In [22]:
ad[~ad.arkhamLabel_name.str.contains('_User', na=False)].dropna(subset=['arkhamLabel_name'])[[
    'address', 'arkhamEntity_name', 'arkhamEntity_type', 'arkhamLabel_name'
]].drop_duplicates(subset=['arkhamEntity_name', 'arkhamEntity_type', 'arkhamLabel_name']).sort_values('arkhamLabel_name')[25:75]

,address,arkhamEntity_name,arkhamEntity_type,arkhamLabel_name
125642,0x98f90bfc702ec548d21b5a566a4df6853e2890d4,Uniswap,dex,AKRO-USDC
204537,0xf3ad3821a0036fd9caec0a4cf5d31515fad03f00,Balancer,dex,AKRO/USDC 70/30 #4
107771,0x8302efa4e838fd15cf09a39f4ca5bff2a3a78443,Balancer,dex,ALBT/OCEAN/USDC 33.33/33.33/33.33
12475,0x0d88b55317516b84e78fbba7cde3f910d5686901,Balancer,dex,ALBT/USDC/ETH 50/25/25
66846,0x5083b16da538c5022744526122243cf3bddb3bf2,Uniswap,dex,ALI-USDC
64066,0x4a14347083b80e5216ca31350a2d21702ac3650d,Tokenlon,dex,AMMWrapperWithPath
191303,0xd3466427f7ee5d0d9cd68b36df677e51bda26321,Balancer,dex,AMP/USDC 80/20
138149,0xb07fe2f407f971125d4eb1977f8acee8846c7324,Uniswap,dex,APE-USDC 2
138990,0xb27c7b131cf4915bec6c4bc1ce2f33f9ee434b9f,SushiSwap,dex,APE-USDT
4210,0x04a3570a08c72fc0e78ed8b18fd3078585443b4c,Uniswap,dex,API3-USDC
